# import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from collections import Counter
import time, datetime, MeCab, random
import pandas as pd

# chromeDriver

In [2]:
#chromeをheadlessで起動
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver91",  chrome_options=options)
#driver = webdriver.Chrome("/usr/local/bin/chromedriver")
top_page = "https://news.yahoo.co.jp/"
dt_now = datetime.datetime.now()
driver.get(top_page)


<ipython-input-2-2040117b6c5e>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver91",  chrome_options=options)


# ニュースリストの取得

In [1]:
df_NL = pd.DataFrame(columns=['news_title','comment_url'])
#topicの選択
select_M_Topic = driver.find_element_by_link_text("トップ").click()
time.sleep(0.1)
select_S_Topic = driver.find_element_by_link_text("主要").click()
time.sleep(0.1)

#ニュース記事への移動
topPage = driver.current_url
for a in range(8):
    section = driver.find_element_by_tag_name("section")
    news_list = section.find_elements_by_tag_name("li")
    mv_news = news_list[a].click()
    time.sleep(0.1)
    news_main = driver.find_element_by_tag_name("article")
    news_title = news_main.find_elements_by_tag_name("p")
    print(news_title[2].text)

#コメントページへの移動
    try:
        news_comment_url = driver.find_element_by_class_name("news-comment-plugin").get_attribute("data-full-page-url")
        df_NL.loc[a] = [news_title[2].text,news_comment_url]    
    except:
        df_NL.loc[a] = [news_title[2],'This news page have no comment.']    
    finally:
        driver.get(topPage) 

dt = dt_now.strftime('%Y%m%d_%H%M')
df_NL.to_csv('NewsList/'+dt+'.csv')

NameError: name 'pd' is not defined

# コメント情報の取得

In [30]:
df_NC = pd.read_csv('NewsList/'+str(dt)+'.csv')
url = df_NC['comment_url'][0]
title = df_NC['news_title'][0]
driver.get(url)
print(title)

#コメント情報の取得

comment_boxes = []

#df = pd.DataFrame(columns=['number','日付','ユーザー名','コメント','good','bad'])
df_NC = pd.DataFrame(columns=['number','ユーザー名','コメント'])
start = 1
end = 200
base_url = driver.current_url
page_url = '?page='
order_url = '&order=older'

iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
driver.switch_to_frame(iframe)
end = driver.find_element_by_class_name("pagenation")
pages = end.find_elements_by_class_name("rapidnofollow")
last_page = pages[7].text
driver.switch_to_default_content()
print(last_page)
for page in range(start,int(last_page)+1):

    driver.get(base_url+page_url+str(page)+order_url) 
    print(driver.current_url)
#    time.sleep(1)
    iframe = driver.find_element_by_class_name("news-comment-plguin-iframe")
    driver.switch_to_frame(iframe)

    comment_boxes = driver.find_elements_by_class_name("root")
    page_comment =0

    for comment_box in comment_boxes:



        page_comment +=1
        comment_number = str(page)+'-'+str(page_comment)

        #ユーザー名取得
        elem_name = comment_box.find_element_by_class_name("rapidnofollow")
        name = elem_name.text

        #コメント取得
        try:
            elem_comment = comment_box.find_element_by_class_name("cmtBody")
        except:
            elem_comment = comment_box.find_element_by_class_name("yjxComment")

        comment = elem_comment.text
        comment.strip("\n")

        #good数取得
        try:
            agree_box = comment_box.find_element_by_class_name("good")
            elem_agree = agree_box.find_element_by_class_name("userNum")
            agree = elem_agree.text
        except:
            pass

        #bad数取得
        try:
            disagree_box = comment_box.find_element_by_class_name("bad")
            elem_disagree = disagree_box.find_element_by_class_name("userNum")
            disagree = elem_disagree.text
        except:
            pass
        #df.loc[(page-1)*10+page_comment] = [comment_number,date, name, comment, agree, disagree]
        df_NC.loc[(page-1)*10+page_comment] = [comment_number, name, comment]

df_NC.to_csv('NewsComment/'+title+".csv")
driver.quit()

In [31]:
print(title)


「ワクチン接種に全力尽くす」　河野氏が会見で抱負　輸送や会場設定など担当


In [3]:
df = pd.read_csv(./, sep=',')

FileNotFoundError: [Errno 2] No such file or directory: './NewsComment/「ワクチン接種に全力尽くす」\u3000河野氏が会見で抱負\u3000輸送や会場設定など担当.csv'

In [32]:
df_parses = pd.DataFrame(columns=['出現単語','原型','品詞'])
wakati = MeCab.Tagger("-Owakati")
chasen = MeCab.Tagger("-Ochasen")
comment_list = df_NC['コメント']
words = []
c=0
print(comment_list[1])
#for b in range(df.shape[0]):
# comment = comment_list[0]
# print(comment)
# words += wakati.parse(comment).split()
# for word in words:
#     Word = chasen.parse(word).split()
#     #print(Word)
#     if '助詞' in Word[3] or '記号' in Word[3]:
#         continue
#     else:
#         df_parses.loc[c]=[Word[0],Word[2],Word[3]]
#     c+=1
# df_parses

ハンコ以上のスピード感に期待したい。


In [7]:
for comment in comment_list:
    wordlist = wakati.parse(comment).split()
    markov = {}
    w1 = ""
    w2 = ""
    for word in wordlist:
        if w1 and w2:
            if (w1, w2) not in markov:
                markov[(w1, w2)]=[]
            markov[(w1, w2)].append(word)
        w1 = w2
        w2 = word
#    print(markov)

{('ハンコ', '以上'): ['の'], ('以上', 'の'): ['スピード感'], ('の', 'スピード感'): ['に'], ('スピード感', 'に'): ['期待'], ('に', '期待'): ['し'], ('期待', 'し'): ['たい'], ('し', 'たい'): ['。']}
{('ゲイツ', 'の'): ['ワクチン'], ('の', 'ワクチン'): ['プログラム'], ('ワクチン', 'プログラム'): ['トランプ大統領'], ('プログラム', 'トランプ大統領'): ['が'], ('トランプ大統領', 'が'): ['武漢'], ('が', '武漢'): ['ウイルス'], ('武漢', 'ウイルス'): ['の'], ('ウイルス', 'の'): ['機密情報'], ('の', '機密情報'): ['解除'], ('機密情報', '解除'): ['か'], ('解除', 'か'): ['・'], ('か', '・'): ['・'], ('・', '・'): ['・']}
{('行政改革', 'に'): ['使う'], ('に', '使う'): ['力'], ('使う', '力'): ['は'], ('力', 'は'): ['ゼロ'], ('は', 'ゼロ'): ['。']}
{('上手く', '行け'): ['ば'], ('行け', 'ば'): ['、'], ('ば', '、'): ['任命'], ('、', '任命'): ['し'], ('任命', 'し'): ['た'], ('し', 'た'): ['ガースー'], ('た', 'ガースー'): ['の'], ('ガースー', 'の'): ['手柄', '人相'], ('の', '手柄'): ['。'], ('手柄', '。'): ['失敗'], ('。', '失敗'): ['し'], ('失敗', 'し'): ['たら'], ('し', 'たら'): ['、'], ('たら', '、'): ['河野'], ('、', '河野'): ['の'], ('河野', 'の'): ['責任'], ('の', '責任'): ['。'], ('責任', '。'): ['正に'], ('。', '正に'): ['、'], ('正に', '、'): ['どちら'], ('、',

In [8]:
driver.quit()

In [44]:
comments = ''
for d in range(1,11):
    comments += comment_list[d]
wordlist = wakati.parse(comments).split()
markov = {}
w1 = ""
w2 = ""
for word in wordlist:
    if w1 and w2:
        if (w1, w2) not in markov:
            markov[(w1, w2)]=[]
        markov[(w1, w2)].append(word)
    w1 = w2
    w2 = word
markov

{('ハンコ', '以上'): ['の'],
 ('以上', 'の'): ['スピード感'],
 ('の', 'スピード感'): ['に'],
 ('スピード感', 'に'): ['期待'],
 ('に', '期待'): ['し', 'し'],
 ('期待', 'し'): ['たい', 'たい'],
 ('し', 'たい'): ['。', '。'],
 ('たい', '。'): ['ゲイツ', 'ワクチン'],
 ('。', 'ゲイツ'): ['の'],
 ('ゲイツ', 'の'): ['ワクチン'],
 ('の', 'ワクチン'): ['プログラム'],
 ('ワクチン', 'プログラム'): ['トランプ大統領'],
 ('プログラム', 'トランプ大統領'): ['が'],
 ('トランプ大統領', 'が'): ['武漢'],
 ('が', '武漢'): ['ウイルス'],
 ('武漢', 'ウイルス'): ['の'],
 ('ウイルス', 'の'): ['機密情報'],
 ('の', '機密情報'): ['解除'],
 ('機密情報', '解除'): ['か'],
 ('解除', 'か'): ['・'],
 ('か', '・'): ['・'],
 ('・', '・'): ['・', '行政改革'],
 ('・', '行政改革'): ['に'],
 ('行政改革', 'に'): ['使う'],
 ('に', '使う'): ['力'],
 ('使う', '力'): ['は'],
 ('力', 'は'): ['ゼロ'],
 ('は', 'ゼロ'): ['。'],
 ('ゼロ', '。'): ['上手く'],
 ('。', '上手く'): ['行け'],
 ('上手く', '行け'): ['ば'],
 ('行け', 'ば'): ['、'],
 ('ば', '、'): ['任命'],
 ('、', '任命'): ['し'],
 ('任命', 'し'): ['た'],
 ('し', 'た'): ['ガースー'],
 ('た', 'ガースー'): ['の'],
 ('ガースー', 'の'): ['手柄', '人相'],
 ('の', '手柄'): ['。'],
 ('手柄', '。'): ['失敗'],
 ('。', '失敗'): ['し'],
 ('失敗', 'し'):

In [34]:
count = 0  # 文のカウンタ
num= 100  # 生成する文の数
sentence = ""
sentences = pd.DataFrame(columns=['自動生成文'])
w1, w2  = random.choice(list(markov.keys())) # 2単語をランダムに選ぶ
while count < num:
    try:
        tmp = random.choice(markov[(w1, w2)])  # 2単語に続く候補の中からランダムに選択する
    except:
        pass
    sentence += tmp  # 文に１単語追加
    if(tmp=='。'):  # '。' があったら
        sentences.loc[count] = [sentence]
        count += 1  # 文のカウンタを＋１
        sentence = ""  # 文を格納する変数
    w1, w2 = w2, tmp  # １単語スライド
sentences.to_csv('Create_text/'+title+'.csv')